# Best-practices for Cloud-Optimized Geotiffs

**Part 1. A single COG**

Let's start simple with a single COG and add complexity as we go. The goal is to get this data into a Python Xarray object as efficiently as possible. We'd like to read the data directly from S3 into Python objects in memory, rather than downloading and operating on a local file.

The test file is a single band image with 5490x5490 pixels. 60MB compressed (120MB uncompressed).

The timing blocks in this notebook are not meant for thorough benchmarking, but it's good to have ballpark numbers on timing. rerunning this notebook a few times shows that most cells doing some sort of network request to S3 have variability on the order of 100ms.

In [ ]:
import os
import xarray as xr
import rasterio
import rioxarray
import hvplot.xarray
import gdal
import requests

In [ ]:
print(xr.__version__)
print(rasterio.__version__)
print(gdal.VersionInfo())

In [ ]:
# path to a COG on S3 (or same file via https url)
s3uri = 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/10/T/ET/2020/S1B_20200106_10TET_ASC/Gamma0_VV.tif'
url = 'https://sentinel-s1-rtc-indigo.s3.us-west-2.amazonaws.com/tiles/RTC/1/IW/10/T/ET/2020/S1B_20200106_10TET_ASC/Gamma0_VV.tif'

In [ ]:
# Note there is no way to tell we have a Cloud-Optimized Geotiff, just from the file name. 
# you can use this nice CLI https://github.com/cogeotiff/rio-cogeo
# or this GDAL script: https://github.com/OSGeo/gdal/blob/master/gdal/swig/python/samples/validate_cloud_optimized_geotiff.py
# Or this API validator from RadiantEarth:
resp = requests.get(f'http://cog-validate.radiant.earth/api/validate?url={url}')
resp.json()['status']

## Download then open

For starters how long does it take to download this file, open it with xarray? This isn't the main use-case of COGs. They are optimized for retrieving a subsets of data from within a file. Nevertheless, we often want to do computations requiring all the pixels, and this is our baseline workflow.

In [ ]:
%%time
# 1.2 - 3.1s
!curl -O {url}
localFile = os.path.basename(url)
da = xr.open_rasterio(localFile)

In [ ]:
print(f'Uncompressed size: {da.nbytes/1e6} MB') # (stored with COMPRESSION=DEFLATE)
da

In [ ]:
%%time 
# 0.8 - 1.2s
da.mean(dim=['x','y'])

In [ ]:
del da #clear cache for next example

**recap:** 
* It takes about 1-2 s to download the entire file (Network bottleneck, running on a laptop-like machine in the same datacenter AWS us-west-2). 
* It takes ~1s to decompresss and read all pixel values into RAM (CPU bottleneck) and compute mean

## Default remote read

If we use `xr.open_rasterio()` we can open the URL directly rather than the remote file path, and the computation time is a bit slower, but about the same as our first test

In [ ]:
%%time
#1 - 1.6s
da = xr.open_rasterio(url)

In [ ]:
%%time
#1.5 - 3s
da.mean(dim=['x','y'])

If you re-run the same command, things are very fast because the data is cached locally

In [ ]:
%%time
# 50-150ms
da = xr.open_rasterio(url)
da.mean(dim=['x','y'])

In [ ]:
del da

**recap:** 

Using *default* settings 
* It takes about 1-3 s just to read the COG into an xarray object via HTTPS. 
* Then an additional 1-3s to stream all pixel values into RAM and compute mean
* Subsequent calls to this dataset are neglible due to caching.

## GDAL environment variables

Why do GDAL environment variables matter? [xr.open_rasterio](http://xarray.pydata.org/en/stable/generated/xarray.open_rasterio.html) uses [rasterio.open](https://github.com/mapbox/rasterio) behind the scenes, which in turn depends on [GDAL](https://github.com/OSGeo/gdal), a foundational piece of geospatial software. Remote reading of data is accomplished with [GDAL's "Virtual File Systems"](https://gdal.org/user/virtual_file_systems.html). Effectively GDAL is translating requests to read from a file into HTTP Range Requests via CURL behind the scenes. 

GDAL has **a lot** of Environment variables that control it's behavior, and these environment variables often default to settings optimized for access to files on your local hard drive. So it turns out changing these settings can have a big impact on the performance of xr.open_rasterio(). Here is a summary of important ones for reading COGS over a network

| Environment Variable | Default | Description |
| ---- | ---- | ---- | 
| GDAL_DISABLE_READDIR_ON_OPEN | YES | scan directory of .tif for sidecar files |
| AWS_NO_SIGN_REQUEST | NO | Send AWS credentials along with the request to S3 |
| VSI_CURL_CACHE_SIZE | 16MB | A global LRU cache of 16 MB shared among all downloaded content is enabled by default |
| GDAL_MAX_RAW_BLOCK_CACHE_SIZE | 10MB | merge consecutive HTTP Byte Range Requests up to X MB |
| GDAL_SWATH_SIZE |  10MB|  Size of the swath when copying raster data from one dataset to another one (in bytes). Should not be smaller than GDAL_CACHEMAX. |

*See these sites for more details*:
* https://gdal.org/user/virtual_file_systems.html#network-based-file-systems
* https://gdal.org/user/configoptions.html 

In [ ]:
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR' #This is KEY! otherwise we send a bunch of HTTP GET requests to test for common sidecar metadata
os.environ['AWS_NO_SIGN_REQUEST']='YES' #Since this is a public bucket, we don't need authentication
os.environ['GDAL_MAX_RAW_BLOCK_CACHE_SIZE']='200000000'  #200MB: Want this to be greater than size of uncompressed raster to overcome a 10 MB limit in the GeoTIFF driver for range request merging.
os.environ['GDAL_SWATH_SIZE']='200000000'  #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE
os.environ['VSI_CURL_CACHE_SIZE']='200000000' #also increase this if increasing MAX_RAW_BLOCK_CACHE_SIZE
# Others?

In [ ]:
%%time
#40-400ms
da = xr.open_rasterio(url)

In [ ]:
%%time
#1 - 3s
da.mean(dim=['x','y'])

In [ ]:
del da

**recap:** 

* performance is now on-par or better than downloading the entire file, opening and processing on local hard drive!
* setting `GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR` vastly improves opening time from ~1.4s to ~100ms!!!
    * in my experience this is as good as it gets, to read metadata necessary to describe the file you request 16kB by default
* if you're reading more than 10MB of data increasing `GDAL_MAX_RAW_BLOCK_CACHE_SIZE`, `GDAL_SWATH_SIZE`, and `VSI_CURL_CACHE_SIZE` can reduce the number of network requests and improve performaence.

## Dask

xarray is also integrated with Dask, so instead of pixel values stored in numpy arrays they are stored as [dask arrays](http://xarray.pydata.org/en/stable/dask.html) suitable for parallel computing. By default, dask uses its [single-machine multi-threaded scheduler](https://docs.dask.org/en/latest/setup/single-machine.html), which distributes work across multiple CPU threads. 

Operations are more transparent if you explicitly initialize your cluster, and use the dask distributed scheduler. This also gives you access to diagnostic info via the dask labextension. In general, stick with default cluster settings. Below we explicitly set `processes=False` because loading a COG into xarray gives us a Dask Array and according to the documentation (https://docs.dask.org/en/latest/setup/single-distributed.html#client):

```
This is sometimes preferable if you want to avoid inter-worker communication and your computations release the GIL. This is common when primarily using NumPy or Dask Array.
```

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(processes=False, local_directory='/tmp') # specify dask worker directory to avoid /home NFS mount
client = Client() 
client #this will give you a url such as /user/scottyhq/proxy/8787/status that goes into the labextension dashboard

In [ ]:
%%time
# ~100 - 700ms
chunks=dict(band=1, x=2745, y=2745) # dask maps each of these subarrays (aka "chunks" or "tiles") to parallel processes or threads
da = xr.open_rasterio(url, chunks=chunks)

In [ ]:
%%time
# ~50 micoseconds!
ave = da.mean(dim=['x','y'])
ave

In [ ]:
# visualize the task graph of this computation
ave.data.visualize(optimize_graph=True, rankdir='LR')

xarray with dask arrays are just a view of the data (known from metadata alone), and a task-graph that maps out tasks needed to run a computation! Now that the pixels are stored as a dask array, we have to explicitly call .compute() to bring the into RAM on our machine:

In [ ]:
%%time
# 1 - 3s 
ave.compute()

**recap:** 

* performance is comparable to downloading the entire file, opening and processing on local hard drive!
* reading the COG into a dask array was ~100ms, same as a numpy array, because in both cases we only have 1 GET request to read metadata
* reading all the data into memory was ~1.6s. Dask adds overhead to manage parallel computations, and in this case we're using 4 threads in parallel to access the data. When working with a single 100MB Dask isn't really necessary, but as we start to explore larger datasets that don't comfortably fit in memory dask will be crucial.

### Visualize

`rioxarray` extends xarray DataArrays with nice geospatial features. In particular:
- direct loading of precomputed TIF overviews
- warping (CRS reprojection) with rasterio gdal
- clipping based on vector data (e.g. Polygon geometries)

hvplot is a great way to interactively explore a COG in the browser:

In [ ]:
%%time
da = rioxarray.open_rasterio(url, masked=True, overview_level=3).squeeze('band')
da.hvplot.image(cmap='gray', aspect='equal', clim=(0,0.4), title=url[64:])